In [79]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/ambazonia.csv


In [80]:
import pandas as pd
ambazonia_textdata = pd.read_csv("../input/ambazonia.csv")
ambazonia_textdata.head(5)

,TIMESTAMP,TWEETS
0,2/17/2020 16:01,"b'@HonWirbaJoseph @HonWirbaJoseph,no true lead..."
1,2/17/2020 15:58,b'RT @DFombat: Internet #BlackOut in Menchum c...
2,2/17/2020 15:55,b'RT @DFombat: Internet #BlackOut in Menchum c...
3,2/17/2020 15:55,b'At least 22 killed in #Cameroon. Fingers poi...
4,2/17/2020 15:51,"b""RT @MumuluhN: #Cameroon #army burnt civilian..."


In [81]:
#lets look at the type of data that we have at hand
type(ambazonia_textdata)

pandas.core.frame.DataFrame

In [82]:
#lets see if it has the same error in naming as the SpeechData
ambazonia_textdata.columns

Index(['  TIMESTAMP', '                                                                                                                                                           TWEETS'], dtype='object')

In [83]:
#restructuring the dataset columns
ambazonia_textdata_altered =ambazonia_textdata.rename(columns={'  TIMESTAMP':'Timestamp', '                                                                                                                                                           TWEETS':'Tweets'})

In [84]:
ambazonia_textdata_altered.head(5)

,Timestamp,Tweets
0,2/17/2020 16:01,"b'@HonWirbaJoseph @HonWirbaJoseph,no true lead..."
1,2/17/2020 15:58,b'RT @DFombat: Internet #BlackOut in Menchum c...
2,2/17/2020 15:55,b'RT @DFombat: Internet #BlackOut in Menchum c...
3,2/17/2020 15:55,b'At least 22 killed in #Cameroon. Fingers poi...
4,2/17/2020 15:51,"b""RT @MumuluhN: #Cameroon #army burnt civilian..."


In [85]:
#checking out if the columns have been altered
ambazonia_textdata_altered.columns

Index(['Timestamp', 'Tweets'], dtype='object')

In [86]:
#checking the type of data
type(ambazonia_textdata_altered)

pandas.core.frame.DataFrame

In [87]:
#lets work on preprocessing the Tweets part
tweets = ambazonia_textdata_altered.Tweets

In [88]:
#checking out the just created Tweets dataset
tweets.head(5)

0    b'@HonWirbaJoseph @HonWirbaJoseph,no true lead...
1    b'RT @DFombat: Internet #BlackOut in Menchum c...
2    b'RT @DFombat: Internet #BlackOut in Menchum c...
3    b'At least 22 killed in #Cameroon. Fingers poi...
4    b"RT @MumuluhN: #Cameroon #army burnt civilian...
Name: Tweets, dtype: object

In [89]:
#importing all necessary dependencies
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import *
import string
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix
import seaborn

#preprocessing the tweets dataset that will be later pseudoencoded

## 1. Removal of punctuation and capitlization
## 2. Tokenizing
## 3. Removal of stopwords
## 4. Stemming

#based on what I have picked up I am going to use French stopwords
stopwords = nltk.corpus.stopwords.words("french")

#extending the stopwords to include other words used in twitter such as retweet(rt) etc.
other_exclusions = ["#ff", "ff", "rt","b"]
stopwords.extend(other_exclusions)
stemmer = PorterStemmer()

def preprocess(tweets):  
    
    # removal of extra spaces
    regex_pat = re.compile(r'\s+')
    tweets_space = tweets.str.replace(regex_pat, ' ')

    # removal of @name[mention]
    regex_pat = re.compile(r'@[\w\-]+')
    tweets_name = tweets_space.str.replace(regex_pat, '')

    # removal of links[https://abc.com]
    giant_url_regex =  re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
            '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    tweets = tweets_name.str.replace(giant_url_regex, '')
    
    # removal of punctuations and numbers
    punc_remove = tweets.str.replace("[^a-zA-Z]", " ")
    # removal of capitalization
    tweets_lower = punc_remove.str.lower()
    
    # tokenizing
    tokenized_tweets = tweets_lower.apply(lambda x: x.split())
    
    # removal of stopwords
    tokenized_tweets=  tokenized_tweets.apply(lambda x: [item for item in x if item not in stopwords])
    
    # stemming of the tweets
    tokenized_tweets = tokenized_tweets.apply(lambda x: [stemmer.stem(i) for i in x]) 
    
    for i in range(len(tokenized_tweets)):
        tokenized_tweets[i] = ' '.join(tokenized_tweets[i])
        tweets_p= tokenized_tweets
    
    return tweets_p

processed_tweets = preprocess(tweets)

ambazonia_textdata_altered['processed_tweets'] = processed_tweets
ambazonia_textdata_altered.head(10)

,Timestamp,Tweets,processed_tweets
0,2/17/2020 16:01,"b'@HonWirbaJoseph @HonWirbaJoseph,no true lead...",no true leader in the ambazonia war of indepen...
1,2/17/2020 15:58,b'RT @DFombat: Internet #BlackOut in Menchum c...,internet blackout in menchum counti ambazonia ...
2,2/17/2020 15:55,b'RT @DFombat: Internet #BlackOut in Menchum c...,internet blackout in menchum counti ambazonia ...
3,2/17/2020 15:55,b'At least 22 killed in #Cameroon. Fingers poi...,at least kill in cameroon finger point to armi...
4,2/17/2020 15:51,"b""RT @MumuluhN: #Cameroon #army burnt civilian...",cameroon armi burnt civilian in ambazonia dure...
5,2/17/2020 15:49,b'RT @wilfriedtabe: #Cameroon. Remember few da...,cameroon rememb few day ago th februari befor ...
6,2/17/2020 15:47,b'RT @ngans15: The escalation of #genocide in ...,the escal of genocid in ambazonia perpetr by t...
7,2/17/2020 15:47,"b""RT @apeterbruce: \xf0\x9f\x87\xa8\xf0\x9f\x8...",xf x f x xa xf x f x xb zombi ha test the weak...
8,2/17/2020 15:36,b'RT @DFombat: Internet #BlackOut in Menchum c...,internet blackout in menchum counti ambazonia ...
9,2/17/2020 15:25,b'RT @AmbaLastborn: The interim govt of the Fe...,the interim govt of the feder rep of ambazonia...


In [90]:
#lets look at whether there is an altering in the dataset type
type(ambazonia_textdata_altered)

pandas.core.frame.DataFrame

In [91]:
#lets see the exclusions that you missed from several rows so that you can get them out
ambazonia_textdata_altered.processed_tweets[0:3]

0    no true leader in the ambazonia war of indepen...
1    internet blackout in menchum counti ambazonia ...
2    internet blackout in menchum counti ambazonia ...
Name: processed_tweets, dtype: object

In [92]:
ambazonia_textdata_altered.processed_tweets.iloc[2]

'internet blackout in menchum counti ambazonia anoth massacr current go we speak xe x xa'

In [93]:
#checking the context of the use of the word concordance
from nltk.text import Text
a = Text(word_tokenize(ambazonia_textdata_altered.processed_tweets.iloc[2]))
a.concordance('internet')

Displaying 1 of 1 matches:
                                   internet blackout in menchum counti ambazon
